# ***SIMPLE CNN MODEL BUILD FOR LUNG CANCER IMAGING***

**Step 1: Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Step 2: Create Directories in Drive**

**Step 3: Install Dependenices**

In [ ]:
!pip install -q -U tensorflow==2.13.0
!pip install -q -U tensorflow-addons==0.23.0
import tensorflow as tf
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import os
import sys
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

project_path = '/content/drive/MyDrive/cnn_project'
sys.path.append(project_path)

**Mini Step 3a: Verify Fixes**

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

print(f"TensorFlow version: {tf.__version__}")
print(f"TensorFlow Addons version: {tfa.__version__}")

**Step 4: Use a script to Load and Preprocess Data**

In [ ]:
import cv2
import numpy as np
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

class DataProcessor:
    def __init__(self, base_path="/content/drive/MyDrive/cnn_project"):
        self.base_path = Path(base_path)
        self.train_path = self.base_path / "data/raw/train"
        self.test_path = self.base_path / "data/raw/test"

    def load_images(self, directory, target_size=(256, 256), use_clahe=False):
        images = []
        labels = []
        class_names = ['benign', 'malignant', 'normal']
        for label, class_name in enumerate(class_names):
            class_dir = directory / class_name
            for img_path in class_dir.glob('*'):
                if img_path.suffix.lower() in ('.png', '.jpg', '.jpeg'):
                    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
                    if img is None:
                        print(f"Warning: Unable to load image {img_path}")
                        continue
                    if use_clahe:
                        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                        img = clahe.apply(img)
                    img = cv2.resize(img, target_size)
                    images.append(img)
                    labels.append(label)
        return np.array(images), np.array(labels)

    def create_generators(self, batch_size=32, use_clahe=False):
        # Load images and normalize to [0, 1]
        X_train, y_train = self.load_images(self.train_path, use_clahe=use_clahe)
        X_train = X_train[..., np.newaxis] / 255.0

        # Split into training and validation sets (stratified)
        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
        )

        # Data augmentation for training
        train_datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            brightness_range=[1.0, 1.5],
            fill_mode='nearest'
        )
        val_datagen = ImageDataGenerator()  # No augmentation for validation

        train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size, shuffle=True)
        val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size, shuffle=False)
        return train_generator, val_generator, X_val, y_val

    def show_augmented_images(self, generator, n_images=5, gamma=1.0):
        """
        Visualizes a few augmented images.
        - gamma: a factor for gamma correction (set gamma < 1 to brighten).
        """
        images, labels = next(generator)
        plt.figure(figsize=(15, 3))
        for i in range(n_images):
            img = images[i].squeeze()
            if gamma != 1.0:
                img = np.power(img, 1.0/gamma)
            plt.subplot(1, n_images, i+1)
            plt.imshow(img, cmap='gray', vmin=0, vmax=1)
            plt.title(f"Label: {labels[i]}")
            plt.axis('off')
        plt.tight_layout()
        plt.show()

**Step 5: Model Development**

In [ ]:
import tensorflow as tf

def create_model(input_shape=(256, 256, 1), num_classes=3):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

**Step 6: Model Training**

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
from utils.data_processing import DataProcessor
from utils.model import create_model

def train_model(model, train_generator, val_generator, epochs=50, checkpoint_path='best_model.h5'):
    # Set up callbacks for robust training
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
    checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
    callbacks = [early_stop, checkpoint, reduce_lr]

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    return history

def plot_training_history(history):
    # Plot loss and accuracy over epochs
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    # Initialize the data processor and generators
    processor = DataProcessor()
    train_gen, val_gen, X_val, y_val = processor.create_generators(batch_size=32)

    # Create the model
    model = create_model(input_shape=(256, 256, 1), num_classes=3)

    # Train the model
    history = train_model(model, train_gen, val_gen, epochs=50, checkpoint_path='best_model.h5')

    # Plot training history
    plot_training_history(history)

    # Optionally, save the final model
    model.save('final_model.h5')

**Step 7: Model Evaluation**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def evaluate_model(model, X_val, y_val, class_names=['benign', 'malignant', 'normal']):
    # Generate predictions and convert to class indices
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Print classification report
    print("Classification Report:")
    print(classification_report(y_val, y_pred_classes, target_names=class_names))

    # Plot confusion matrix
    cm = confusion_matrix(y_val, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    # Print per-class and overall accuracy
    class_acc = {}
    for i, class_name in enumerate(class_names):
        mask = (y_val == i)
        class_acc[class_name] = np.mean(y_pred_classes[mask] == y_val[mask])
    print("Class-wise Accuracy:")
    for class_name, acc in class_acc.items():
        print(f"{class_name}: {acc:.2%}")
    overall_acc = accuracy_score(y_val, y_pred_classes)
    print(f"Overall Accuracy: {overall_acc:.2%}")

In [ ]:
# Run the evaluation function
from utils.evaluate import evaluate_model
evaluate_model(model, X_val, y_val)

ModuleNotFoundError: No module named 'utils'

**Step 8: Implementation of Explainable AI (Grad-CAM)**

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt

def grad_cam(model, img_array, layer_name='conv2d_2', class_index=None):
    # Create a model that outputs the target convolutional layer and predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if class_index is None:
            class_index = tf.argmax(predictions[0])
        loss = predictions[:, class_index]

    # Compute gradients and perform global pooling
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap).numpy()
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) if np.max(heatmap) != 0 else 1

    # Resize the heatmap and apply the color map
    heatmap = cv2.resize(heatmap, (img_array.shape[2], img_array.shape[1]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Superimpose the heatmap on the original image (converted from grayscale)
    img = np.uint8(255 * img_array[0].squeeze())
    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    superimposed_img = cv2.addWeighted(img_bgr, 0.6, heatmap, 0.4, 0)
    return superimposed_img

def visualize_explanations(model, X_val, y_val, num_samples=3):
    class_names = ['benign', 'malignant', 'normal']
    indices = np.random.choice(len(X_val), num_samples, replace=False)
    plt.figure(figsize=(15, 5 * num_samples))
    for i, idx in enumerate(indices, 1):
        img = X_val[idx]
        true_label = class_names[y_val[idx]]
        pred = model.predict(np.expand_dims(img, axis=0))
        pred_class = class_names[np.argmax(pred)]
        explanation = grad_cam(model, np.expand_dims(img, axis=0))

        plt.subplot(num_samples, 2, 2 * i - 1)
        plt.imshow(img.squeeze(), cmap='gray')
        plt.title(f"True: {true_label}\nPred: {pred_class}")
        plt.axis('off')

        plt.subplot(num_samples, 2, 2 * i)
        plt.imshow(explanation)
        plt.title("Grad-CAM Explanation")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize model explanations

from utils.xai import visualize_explanations
visualize_explanations(model, X_val, y_val, num_samples=3)

**Step 9: Running the Project**

In [ ]:
import sys
project_path = '/content/drive/MyDrive/cnnprojectfinal'
if project_path not in sys.path:
    sys.path.append(project_path)

**Step 10: Testing Model Accuracy on Test images**

In [ ]:
def load_test_images(test_dir, target_size=(256, 256)):
    test_dir = Path(test_dir)
    images = []
    image_paths = list(test_dir.glob('*'))
    for img_path in image_paths:
        if img_path.suffix.lower() in ('.png', '.jpg', '.jpeg'):
            img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, target_size)
                images.append(img[..., np.newaxis] / 255.0)
    return np.array(images), image_paths

# Usage:
test_images, test_paths = load_test_images("/content/drive/MyDrive/cnnprojectfinal/data/raw/test")
predictions = model.predict(test_images)